# Table of Contents
 <p><div class="lev1 toc-item"><a href="#Overview" data-toc-modified-id="Overview-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Overview</a></div><div class="lev2 toc-item"><a href="#Input" data-toc-modified-id="Input-11"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Input</a></div><div class="lev2 toc-item"><a href="#Save-the-df" data-toc-modified-id="Save-the-df-12"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Save the df</a></div>

In [1]:
%pylab inline
import pandas as pd

Populating the interactive namespace from numpy and matplotlib


# Overview

In this notebook we show how to import the ISTAT commuting data to provide information on commuting to the synthetic population generator.

We are going to load the tables containing the information about the number of students/workers commuting to:
- same municipality of residence;
- different municipality in same province of residence;
- different province in same region of residence;
- different region in same nation.
- by now we will ignore the inter-state commuting as it is marginal.
- also, we assume that all the people working and/or studying do commute to a workplace/school thus ignoring people working at home (even thoughdata are available).

We will then convert these absolute counts to commuting probabilities and project them to all the eurostat codes to create a fake commuting matrix for teh whole Europe.

## Input

Here we import the input files (excel tables) downloaded from the [ISTAT](http://dati-censimentopopolazione.istat.it/index.aspx?queryid=3432) website.

In [3]:
columnCodes = ["NUTS", "sCty", "sMun", "sMun_CT",
               "dMun_sPrv", "dMun_sPrv_CT",
               "dPrv_sReg", "dPrv_sReg_CT",
               "dReg", "dReg_CT", "sCty_CT",
               "dCty", "TOTAL"
              ]

studentsCommuting_df = pd.read_excel("resources/Italy/population/commuting_studentsCommuters_ISTAT_2011_NUTS3.xlsx",
                                    na_values="..", skiprows=3, names=columnCodes)

workersCommuting_df = pd.read_excel("resources/Italy/population/commuting_workersCommuters_ISTAT_2011_NUTS3.xlsx",
                                    na_values="..", skiprows=3, names=columnCodes)

nonCommuting_df = pd.read_excel("resources/Italy/population/commuting_nonCommuters_ISTAT_2011_NUTS3.xlsx",
                                    na_values="..", skiprows=1,
                                    names=["NUTS", "studyHome", "workHome", "workNoPlace", "NEET", "TOTAL"])

# Replace holes with no commuters...
workersCommuting_df.replace(to_replace=np.NaN, value=0., inplace=True)
studentsCommuting_df.replace(to_replace=np.NaN, value=0., inplace=True)
nonCommuting_df.replace(to_replace=np.NaN, value=0., inplace=True)

# Drop columns about the destination in capitals (we do not care at
# this stage to distinguish between commuting to capital or other
# municipalities)...
for capitalColumn in [c for c in columnCodes if "_CT" in c]:
    del workersCommuting_df[capitalColumn]
    del studentsCommuting_df[capitalColumn]

# Remove spaces in nuts codes and keep only up to NUTS3 (no LAU1).
workersCommuting_df["NUTS"] = workersCommuting_df["NUTS"].apply(lambda v: v.replace(" ", ""))
studentsCommuting_df["NUTS"] = studentsCommuting_df["NUTS"].apply(lambda v: v.replace(" ", ""))

workersCommuting_df = workersCommuting_df[workersCommuting_df["NUTS"].str.len() <= 5]
studentsCommuting_df = studentsCommuting_df[studentsCommuting_df["NUTS"].str.len() <= 5]

# Set the index
workersCommuting_df.set_index("NUTS", inplace=True)
studentsCommuting_df.set_index("NUTS", inplace=True)
nonCommuting_df.set_index("NUTS", inplace=True)

In [4]:
workersCommuting_df.head(2)

,sCty,sMun,dMun_sPrv,dPrv_sReg,dReg,dCty,TOTAL
NUTS,,,,,,,
IT,19094609,10311208,7039991,1463557.0,279853.0,63938.0,19158547
ITC,5709067,2454305,2537972,616197.0,100593.0,54813.0,5763880


In [5]:
studentsCommuting_df.head(2)

,sCty,sMun,dMun_sPrv,dPrv_sReg,dReg,dCty,TOTAL
NUTS,,,,,,,
IT,9692150,7171072,2050686,393469.0,76923.0,2024.0,9694174
ITC,2468391,1688017,623733,132194.0,24447.0,1371.0,2469762


In [6]:
nonCommuting_df.head(2)

,studyHome,workHome,workNoPlace,NEET,TOTAL
NUTS,,,,,
IT,651992,968741,1643512,24458090,27722335
ITC,106752,272654,445230,6071110,6895746


In [7]:
# Check that I have the right number of commuters (or a number not too much wrong)
aggregationsToCheck = {
                "sCty": ["sMun", "dMun_sPrv", "dPrv_sReg", "dReg"],
                "TOTAL": ["sCty", "dCty"],
                      }

for dfName, df in zip(["work", "stud"], [workersCommuting_df, studentsCommuting_df]):
    print dfName
    for tot, cols in aggregationsToCheck.iteritems():
        print "\t", tot
        problematic = df[(df[cols].sum(axis=1) != df[tot])]
        if problematic.shape[0] > 0:
            print (problematic[cols].sum(axis=1) - problematic[tot])

work
	TOTAL
	sCty
NUTS
ITF     1.0
ITF3    1.0
dtype: float64
stud
	TOTAL
	sCty


In [8]:
nonCommuting_df[nonCommuting_df[nonCommuting_df.columns[:-1]].sum(axis=1) != nonCommuting_df["TOTAL"]]

,studyHome,workHome,workNoPlace,NEET,TOTAL
NUTS,,,,,


In [9]:
# Check that we have the same indexes
print studentsCommuting_df.index.symmetric_difference(workersCommuting_df.index)

Index([], dtype='object', name=u'NUTS')


In [10]:
# Create fake tables for all the europe NUTS3 sampling from this dataframe...

# Load the reference df
referenceDF = pd.read_pickle("resources/Europe/population/structure/dataframes/1990-2016_broadAgeBySexNUTS3.pkl")

missingNUTS3codes = referenceDF.index.difference(studentsCommuting_df.index)
NmissingNUTS3codes = missingNUTS3codes.shape[0]

In [11]:
missingStudentCommuting = studentsCommuting_df.sample(NmissingNUTS3codes, replace=True)
missingStudentCommuting.index = missingNUTS3codes

europeStudentsCommuting_df = studentsCommuting_df.append(missingStudentCommuting,
                                                   ignore_index=False, verify_integrity=True)
europeStudentsCommuting_df.head()

,sCty,sMun,dMun_sPrv,dPrv_sReg,dReg,dCty,TOTAL
IT,9692150,7171072,2050686,393469.0,76923.0,2024.0,9694174
ITC,2468391,1688017,623733,132194.0,24447.0,1371.0,2469762
ITC1,649789,450189,171848,19675.0,8077.0,54.0,649843
ITC11,346128,252448,90865,2395.0,420.0,14.0,346142
ITC12,24112,15525,5423,2840.0,324.0,0.0,24112


In [12]:
missingWorkerCommuting = workersCommuting_df.sample(NmissingNUTS3codes, replace=True)
missingWorkerCommuting.index = missingNUTS3codes

europeWorkersCommuting_df = workersCommuting_df.append(missingWorkerCommuting,
                                                   ignore_index=False, verify_integrity=True)
europeWorkersCommuting_df.head()

,sCty,sMun,dMun_sPrv,dPrv_sReg,dReg,dCty,TOTAL
IT,19094609,10311208,7039991,1463557.0,279853.0,63938.0,19158547
ITC,5709067,2454305,2537972,616197.0,100593.0,54813.0,5763880
ITC1,1532970,715934,707481,71051.0,38504.0,5700.0,1538670
ITC11,787579,371270,394670,15496.0,6143.0,127.0,787706
ITC12,60985,28827,18534,11894.0,1730.0,13.0,60998


In [13]:
# Now delete columns on foreign commuting and compute the probabilities...
# We will use the counter to the same country as the total value
workersCommutingRAW_df = europeWorkersCommuting_df.copy(deep=True)
studentsCommutingRAW_df = europeStudentsCommuting_df.copy(deep=True)

del workersCommutingRAW_df["dCty"], workersCommutingRAW_df["TOTAL"]
del studentsCommutingRAW_df["dCty"], studentsCommutingRAW_df["TOTAL"]

# Compute probability and delete total...
workersCommutingPDF_df = workersCommutingRAW_df.div(workersCommutingRAW_df["sCty"], axis=0)
studentsCommutingPDF_df = studentsCommutingRAW_df.div(studentsCommutingRAW_df["sCty"], axis=0)

del workersCommutingRAW_df["sCty"], studentsCommutingRAW_df["sCty"]
del workersCommutingPDF_df["sCty"], studentsCommutingPDF_df["sCty"]

# Compute the actual PDF now that we have the total (total may be different from sum of columns)
workersCommutingPDF_df = workersCommutingPDF_df.div(workersCommutingPDF_df.sum(axis=1), axis=0)
studentsCommutingPDF_df = studentsCommutingPDF_df.div(studentsCommutingPDF_df.sum(axis=1), axis=0)

workersCommutingCDF_df = workersCommutingPDF_df.cumsum(axis=1)
studentsCommutingCDF_df = studentsCommutingPDF_df.cumsum(axis=1)

In [14]:
workersCommutingRAW_df.head(2)

,sMun,dMun_sPrv,dPrv_sReg,dReg
IT,10311208,7039991,1463557.0,279853.0
ITC,2454305,2537972,616197.0,100593.0


In [15]:
workersCommutingPDF_df.head(2)

,sMun,dMun_sPrv,dPrv_sReg,dReg
IT,0.540006,0.368690,0.076648,0.014656
ITC,0.429896,0.444551,0.107933,0.017620


In [16]:
workersCommutingCDF_df.head(2)

,sMun,dMun_sPrv,dPrv_sReg,dReg
IT,0.540006,0.908696,0.985344,1.0
ITC,0.429896,0.874447,0.982380,1.0


In [17]:
# concat the three ones!
studentCommutingDistribution = pd.concat({
                                    "RAW": studentsCommutingRAW_df,
                                    "PDF": studentsCommutingPDF_df,
                                    "CDF": studentsCommutingCDF_df,
                                        }).unstack(0)
studentCommutingDistribution.columns = studentCommutingDistribution.columns.swaplevel(1, 0)
studentCommutingDistribution.sort_index(axis=1, level=-2, inplace=True)

In [18]:
studentCommutingDistribution.head(2)

CDF                                PDF                      \
         sMun dMun_sPrv dPrv_sReg dReg      sMun dMun_sPrv dPrv_sReg   
IT   0.739885  0.951467  0.992063  1.0  0.739885  0.211582  0.040597   
ITC  0.683853  0.936541  0.990096  1.0  0.683853  0.252688  0.053555   

                     RAW                                
         dReg       sMun  dMun_sPrv dPrv_sReg     dReg  
IT   0.007937  7171072.0  2050686.0  393469.0  76923.0  
ITC  0.009904  1688017.0   623733.0  132194.0  24447.0

In [19]:
# concat the three ones!
workerCommutingDistribution = pd.concat({
                                    "RAW": workersCommutingRAW_df,
                                    "PDF": workersCommutingPDF_df,
                                    "CDF": workersCommutingCDF_df,
                                        }).unstack(0)
workerCommutingDistribution.columns = workerCommutingDistribution.columns.swaplevel(0, 1)
workerCommutingDistribution.sort_index(axis=1, level=-2, inplace=True)

In [20]:
workerCommutingDistribution.head(2)

CDF                                PDF                      \
         sMun dMun_sPrv dPrv_sReg dReg      sMun dMun_sPrv dPrv_sReg   
IT   0.540006  0.908696  0.985344  1.0  0.540006  0.368690  0.076648   
ITC  0.429896  0.874447  0.982380  1.0  0.429896  0.444551  0.107933   

                      RAW                                  
         dReg        sMun  dMun_sPrv  dPrv_sReg      dReg  
IT   0.014656  10311208.0  7039991.0  1463557.0  279853.0  
ITC  0.017620   2454305.0  2537972.0   616197.0  100593.0

## Save the df

In [26]:
workerCommutingDistribution.to_pickle("resources/Europe/population/structure/dataframes/2011_workCommuting_ISTAT_NUTS3.pkl")
studentCommutingDistribution.to_pickle("resources/Europe/population/structure/dataframes/2011_studyCommuting_ISTAT_NUTS3.pkl")